# **Final Week Bima Aristo**

A saliency map shows the pixels which greatly impacts the classification of an image.
- This is done by getting the gradient of the loss with respect to changes in the pixel values, then plotting the results.
- From there, you can see if your model is looking at the correct features when classifying an image.
  - For example, if you're building a dog breed classifier, you should be wary if your saliency map shows strong pixels outside the dog itself (e.g. sky, grass, dog house, etc...).


### Download test files and weights

Let's begin by first downloading files we will be using for this lab.

In [ ]:
# Download the same test files from the Cats vs Dogs ungraded lab
!wget -O cat1.jpg https://storage.googleapis.com/tensorflow-1-public/tensorflow-3-temp/MLColabImages/cat1.jpeg
!wget -O cat2.jpg https://storage.googleapis.com/tensorflow-1-public/tensorflow-3-temp/MLColabImages/cat2.jpeg
!wget -O catanddog.jpg https://storage.googleapis.com/tensorflow-1-public/tensorflow-3-temp/MLColabImages/catanddog.jpeg
!wget -O dog1.jpg https://storage.googleapis.com/tensorflow-1-public/tensorflow-3-temp/MLColabImages/dog1.jpeg
!wget -O dog2.jpg https://storage.googleapis.com/tensorflow-1-public/tensorflow-3-temp/MLColabImages/dog2.jpeg

# Download prepared weights
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1kipXTxesGJKGY1B8uSPRvxROgOH90fih' -O 0_epochs.h5
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1oiV6tjy5k7h9OHGTQaf0Ohn3FmF-uOs1' -O 15_epochs.h5


### Import the required packages

Please import:

  * Tensorflow
  * Tensorflow Datasets
  * Numpy
  * Matplotlib's PyPlot
  * Keras Models API classes you will be using
  * Keras layers you will be using
  * OpenCV (cv2)

In [ ]:
# YOUR CODE HERE
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
import cv2

### Download and prepare the dataset.



#### Load Cats vs Dogs

* Required: Use Tensorflow Datasets to fetch the `cats_vs_dogs` dataset.
  * Use the first 80% of the *train* split of the said dataset to create your training set.
  * Set the `as_supervised` flag to create `(image, label)` pairs.
    

In [ ]:
# Load the data and create the train set (optional: val and test sets)

# YOUR CODE HERE
tfds.disable_progress_bar()

splits = ["train[:80%]", "train[80%:90%]", "train[90%:]"]

splits, info = tfds.load(name="cats_vs_dogs", with_info=True, as_supervised=True, split=splits)

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits["train"].num_examples
num_classes = info.features["label"].num_classes

num_examples, num_classes

#### Create preprocessing function

Define a function that takes in an image and label. This will:
  * cast the image to float32
  * normalize the pixel values to [0, 1]
  * resize the image to 300 x 300


In [ ]:
IMG_SIZE = (300, 300)
def augmentimages(image, label):
  # YOUR CODE HERE
  image = tf.cast(image, dtype=tf.float32)
  image /= 255.0
  image = tf.image.resize(image, IMG_SIZE)
  return image, label

#### Preprocess the training set

Use the `map()` and pass in the method that is just defined to preprocess the training set.


In [ ]:
augmented_training_data = train_examples.map(augmentimages)

#### Create batches of the training set.

```Python
# Shuffle the data if you're working on your own personal project
train_batches = augmented_training_data.shuffle(1024).batch(32)
```

In [ ]:
train_batches = augmented_training_data.batch(32)

### Build the Cats vs Dogs classifier

Model

In [ ]:
# YOUR CODE HERE
model = tf.keras.Sequential()
model.add(layers.Conv2D(filters=16, kernel_size=(3, 3),
                        input_shape=(IMG_SIZE+(3,)),activation="relu", padding="same"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(128, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(2, activation="softmax"))

model.summary()

**Expected Output:**

```txt
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d (Conv2D)              (None, 300, 300, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 150, 150, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 37, 37, 128)       73856     
_________________________________________________________________
global_average_pooling2d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
=================================================================
Total params: 97,698
Trainable params: 97,698
Non-trainable params: 0
_________________________________________________________________
```

### Create a function to generate the saliency map

Saliency Map Function the `do_salience()`

In [ ]:
from tensorflow.python.framework.importer import import_graph_def
def do_salience(image, model, label, prefix):
  '''
  Generates the saliency map of a given image.

  Args:
    image (file) -- picture that the model will classify
    model (keras Model) -- your cats and dogs classifier
    label (int) -- ground truth label of the image
    prefix (string) -- prefix to add to the filename of the saliency map
  '''

  img = cv2.imread(image)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, IMG_SIZE) / 255.0

  tensor_image = np.expand_dims(img, axis=0)
  number_of_classes = num_classes
  one_hot_label = tf.one_hot([label] * tensor_image.shape[0], num_classes)


  with tf.GradientTape() as tape:
    inputs = tf.cast(tensor_image, dtype=tf.float32)

    # watch the input pixels
    tape.watch(inputs)

    predictions = model(inputs)
    loss = tf.keras.losses.categorical_crossentropy(one_hot_label, predictions)


  gradients = tape.gradient(loss, inputs)
  grayscale_tensor = tf.reduce_sum(tf.abs(gradients), axis=-1)
  normalized_tensor = 255 * (grayscale_tensor - tf.reduce_min(grayscale_tensor)) / (tf.reduce_max(grayscale_tensor) - tf.reduce_min(grayscale_tensor))
  normalized_tensor = tf.cast(normalized_tensor, dtype=tf.uint8)
  normalized_tensor = tf.squeeze(normalized_tensor)


  # plot the normalized tensor
  plt.figure(figsize=(8, 8))
  plt.axis('off')
  plt.imshow(normalized_tensor, cmap='gray')
  plt.show()

  # saved
  salient_image_name = prefix + image
  normalized_tensor = tf.expand_dims(normalized_tensor, -1)
  normalized_tensor = tf.io.encode_jpeg(normalized_tensor, quality=100, format='grayscale')
  writer = tf.io.write_file(salient_image_name, normalized_tensor)

### Generate saliency maps with untrained model


Applying the `do_salience()` function on the following image files:

* `cat1.jpg`
* `cat2.jpg`
* `catanddog.jpg`
* `dog1.jpg`
* `dog2.jpg`

Cats will have the label `0` while dogs will have the label `1`.

In [ ]:
# load initial weights
model.load_weights('0_epochs.h5')

# generate the saliency maps for the 5 test images
# YOUR CODE HERE
do_salience('cat1.jpg', model, 0, 'epoch0_salient')
do_salience('cat2.jpg', model, 0, 'epoch0_salient')
do_salience('catanddog.jpg', model, 0, 'epoch0_salient')
do_salience('dog1.jpg', model, 1, 'epoch0_salient')
do_salience('dog2.jpg', model, 1, 'epoch0_salient')

With untrained weights, we will see something like this in the output.
- we will see strong pixels outside the cat that the model uses that when classifying the image.
- After training that these will slowly start to localize to features inside the pet.

<img src='https://drive.google.com/uc?export=view&id=1h5wP52lwbBUMVLlsgyb-tQl_I9eu42X7' alt='saliency'>


### Configure the model for training

In [ ]:
# YOUR CODE HERE
rmsprop = RMSprop(learning_rate=0.001)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = rmsprop,
    metrics=["accuracy"]
)

In [ ]:
# load pre-trained weights
model.load_weights('15_epochs.h5')

# train the model for just 3 epochs
# YOUR CODE HERE
model.fit(train_batches,
          epochs=3)

### Generate saliency maps at 18 epochs

We use your `do_salience()` function again on the same test images.

In [ ]:
# YOUR CODE HERE
do_salience('cat1.jpg', model, 0, 'salient')
do_salience('cat2.jpg', model, 0, 'salient')
do_salience('catanddog.jpg', model, 0, 'salient')
do_salience('dog1.jpg', model, 1, 'salient')
do_salience('dog2.jpg', model, 1, 'salient')

### Zip the images for grading

Zip the normalized tensor images you generated at 18 epochs

* salientcat1.jpg
* salientcat2.jpg
* salientcatanddog.jpg
* salientdog1.jpg
* salientdog2.jpg

In [ ]:
from zipfile import ZipFile

!rm images.zip

filenames = ['cat1.jpg', 'cat2.jpg', 'catanddog.jpg', 'dog1.jpg', 'dog2.jpg']

# writing files to a zipfile
with ZipFile('images.zip','w') as zip:
  for file in filenames:
    zip.write('salient' + file)

print("images.zip generated!")